In [ ]:
import pandas as pd
import numpy as np
import warnings
from pmdarima.arima import auto_arima
import matplotlib.pyplot as plt

warnings.simplefilter("ignore")

filename = input("Filename: ")
data = pd.read_csv(filename)
params = []

for i in range(0,int(len(data)/24),9):
    d_ =  data[i*24:24*(i+9)]

    d = d_[24:24*(i+9)]
    exogenous_train = pd.DataFrame(d_[0:48])
    train_temp = exogenous_train['Temp_Box'] - np.mean(exogenous_train['Temp_Box']) 
    exogenous_temp = np.concatenate((np.linspace(0,48-1,num=48,dtype = np.uint8).reshape(-1,1),
                                     np.array(train_temp).reshape(-1,1)),axis = 1).reshape(48,-1)
    model = auto_arima(d_[0:48]['Temp'] - np.mean(d_[0:48]['Temp']), X = exogenous_temp, start_p = 1, max_p = 1, start_q = 0, 
                       max_q = 0, max_P = 2, start_Q = 0, max_Q = 0, seasonal = True)

    model.fit(d_[0:48]['Temp']- np.mean(d_[0:48]['Temp']), X = exogenous_temp)
    exogenous_train = pd.DataFrame(d)
    test_temp = exogenous_train['Temp_Box'] - np.mean(exogenous_train['Temp_Box']) 

    exogenous_temp = np.concatenate((np.linspace(0,24*7-1,num=24*7,dtype = np.uint8).reshape(-1,1),
                                     np.array(test_temp).reshape(-1,1)),axis = 1).reshape(24*7,-1)
    forecast = model.predict(24*7, X = exogenous_temp)
    for a in d_[0:48]['Temp']:
        print(a)
    for a in range(len(forecast)):
        print(forecast.values[a] + np.mean(d_[0:24]['Temp']))